*Setting up the enviroment*

In [1]:
!git clone -b stocastic_events https://github.com/zach401/acnportal.git
!pip install acnportal/.
!wget https://ev.caltech.edu/assets/data/gmm/jpl_weekday_40.pkl

fatal: destination path 'acnportal' already exists and is not an empty directory.
Processing ./acnportal
  Created wheel for acnportal: filename=acnportal-0.1.3-cp36-none-any.whl size=81621 sha256=2070e5d3ee4995d83bea8a312a4c202e63cc4542ea3c0d095db33f74239b999d
  Stored in directory: /tmp/pip-ephem-wheel-cache-g8dqiepf/wheels/6d/6a/19/10aef74a8c705c23f53e3e1d696420b07fcdbc88af47701336
Successfully built acnportal
  Found existing installation: acnportal 0.1.3
    Uninstalling acnportal-0.1.3:
      Successfully uninstalled acnportal-0.1.3
--2020-04-16 18:26:22--  https://ev.caltech.edu/assets/data/gmm/jpl_weekday_40.pkl
Resolving ev.caltech.edu (ev.caltech.edu)... 131.215.140.211
Connecting to ev.caltech.edu (ev.caltech.edu)|131.215.140.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10845 (11K) [application/octet-stream]
Saving to: ‘jpl_weekday_40.pkl.1’

jpl_weekday_40.pkl. 100%[===================>]  10.59K  --.-KB/s    in 0s      

2020-04-16 18:26:23 


# Comparing Infrastructure Designs using ACN-Sim
### by Zachary Lee
#### Last updated: 4/16/2020


In this case study, we demonstrate how ACN-Data and ACN-Sim can be used to evaluate infrastructure configurations and algorithms. We consider the case of a site host who expects to charge approximately 100 EVs per day with a demand pattern similar to that of JPL.

The site host has several options, including  
*   102 Uncontrolled Level-1 EVSEs with a 200 kW Transformer
*   30 Uncontrolled Level-2 EVSEs with a 200 kW Transformer
*   102 Uncontrolled Level-2 EVSEs with a 670 kW Transformer
*   102 Smart Level-2 EVSEs running LLF with a 200 kW Transformer

We evaluate the scenarios on the number of times drivers would have to swap parking places to allow other drivers to charge, the percentage of total demand met, and the operating costs (calculated using ACN-Sim's integration with utility tariffs). This demonstrates the significant benefits of developing smart EV charging systems in terms of reducing both capital costs (transformer capacity) and operating costs.


In [0]:
import acnportal

from copy import deepcopy
import warnings
import pytz
import numpy as np
import pickle
from datetime import datetime
from acnportal import acnsim
from acnportal import algorithms
from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff

## Charging Network Designs

To define our charging network options, we will use two functions which generate an AffinityChargingNetwork object. The AffinityChargingNetwork assigns users to spaces dynamically based on available spaces and user preferences. In this example, we will assume each driver has equal preference for all spots.

If all spaces are taken, drivers join a queue which is drained as drivers finish charging and move their vehicle (the early departure option specifies that drivers move their vehicle when it is done charging rather than their normal departure time). We record each time that the user leave and is replaced with someone from the queue as a swap. Swaps are undesirable as they waste time and are frustrating for users. Despite this, swapping is a common practice in many charging facilities where the number of users exceeds the number of EVSEs.  

In [0]:
def level_1_network(transformer_cap=200, evse_per_phase=34):
    """ Configurable charging network for level-1 EVSEs connected line to ground
        at 120 V. 

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of 
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.  
    """
    network = acnsim.network.AffinityChargingNetwork(early_departure=True)
    voltage = 120

    # Define the sets of EVSEs in the Caltech ACN.
    A_ids = ['A-{0}'.format(i) for i in range(evse_per_phase)]
    B_ids = ['B-{0}'.format(i) for i in range(evse_per_phase)]
    C_ids = ['C-{0}'.format(i) for i in range(evse_per_phase)]

    # Add Caltech EVSEs
    for evse_id in A_ids:
        network.register_evse(acnsim.FiniteRatesEVSE(evse_id, [0, 16]), voltage, 0)
    for evse_id in B_ids:
        network.register_evse(acnsim.FiniteRatesEVSE(evse_id, [0, 16]), voltage, 120)
    for evse_id in C_ids:
        network.register_evse(acnsim.FiniteRatesEVSE(evse_id, [0, 16]), voltage, -120)

    # Add Caltech Constraint Set
    I3a = acnsim.Current(A_ids)
    I3b = acnsim.Current(B_ids)
    I3c = acnsim.Current(C_ids)

    # Define intermediate currents
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 277
    secondary_side_constr = transformer_cap * 1000 / 3 / 120
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network


def level_2_network(transformer_cap=200, evse_per_phase=34):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 208 V. 

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of 
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.  
    """
    network = acnsim.network.AffinityChargingNetwork(early_departure=True)
    voltage = 208
    evse_type = 'AeroVironment'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(evse_per_phase)]
    BC_ids = ['BC-{0}'.format(i) for i in range(evse_per_phase)]
    CA_ids = ['CA-{0}'.format(i) for i in range(evse_per_phase)]

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 277
    secondary_side_constr = transformer_cap * 1000 / 3 / 120
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network


## Experiments

In these experiments we will run a simulation for each system configuration can compare the results on key metrics.

In [0]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 208  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 6.6 # kW

**Network Options**

In [0]:
level_1 = level_1_network(transformer_cap=200, evse_per_phase=34)
level_2_200kW_30 = level_2_network(transformer_cap=200, evse_per_phase=10)
level_2_200kW_102 = level_2_network(transformer_cap=200, evse_per_phase=34)
level_2_670kW_102 = level_2_network(transformer_cap=670, evse_per_phase=34)

**Events**

We assume that our site will have a usage profile similar to JPL, so we use a Gaussian Mixture Model trained on data from weekdays at JPL to generate events for this experiment. We assume that the site will be closed on weekends, so no charging will occur. 

In [8]:
gmm = pickle.load(open('jpl_weekday_40.pkl', 'rb'))
sessions_per_day = 100

# Generate a list of the number of sessions to draw for each day.
num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
          [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

# Note that because we are drawing from a distribution, some sessions will be
# invalid, we ignore these sessions and remove the corresponding plugin events. 
synth_events = acnsim.events.gmm_events(gmm, num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)

Invalid session.
Invalid session.
Invalid session.
Invalid session.
Invalid session.
Invalid session.
Invalid session.
Invalid session.
Invalid session.


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.mixture.gaussian_mixture module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.mixture. Anything that cannot be imported from sklearn.mixture is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
def run_experiment(network, algorithm, events):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events. 
    """
    # Timezone of the ACN we are using.
    timezone = pytz.timezone('America/Los_Angeles')
    
    # Start and End times are used when collecting data.
    start = timezone.localize(datetime(2019, 6, 1))
    end = timezone.localize(datetime(2019, 7, 1))
    
    sch = deepcopy(algorithm)
    cn = deepcopy(network)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = acnsim.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sim.run()

    r = {'proportion_of_energy_delivered': acnsim.proportion_of_energy_delivered(sim),
         'energy_delivered': sum(ev.energy_delivered for ev in sim.ev_history.values()),
         'num_swaps': cn.swaps,
         'num_never_charged': cn.never_charged,
         'energy_cost': acnsim.energy_cost(sim),
         'demand_charge': acnsim.demand_charge(sim)
         }
    r['total_cost'] = r['energy_cost'] + r['demand_charge']
    r['$/kWh'] = r['total_cost'] / r['energy_delivered']
    return r

**Define Algorithms**

In [0]:
uncontrolled = algorithms.UncontrolledCharging()
llf = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

**Run Experiments**

In [0]:
level1_uncontrolled = run_experiment(level_1, uncontrolled, 
                                     deepcopy(synth_events))

In [0]:
level2_200kW_uncontrolled = run_experiment(level_2_200kW_30, uncontrolled,
                                           deepcopy(synth_events))

In [0]:
level2_670kW_uncontrolled = run_experiment(level_2_670kW_102, uncontrolled,
                                           deepcopy(synth_events))

In [0]:
level2_200kW_llf = run_experiment(level_2_200kW_102, llf,
                                  deepcopy(synth_events))

### Analyze Results

In [16]:
import pandas as pd
pd.DataFrame({'Level 1: Unctrl: 200 kW : 102 EVSEs': level1_uncontrolled,
              'Level 2: Unctrl: 200 kW : 30 EVSEs':  level2_200kW_uncontrolled,
              'Level 2: Unctrl: 670 kW : 102 EVSEs': level2_670kW_uncontrolled,
              'Level 2: LLF: 200 kW : 102 EVSEs': level2_200kW_llf})

,Level 1: Unctrl: 200 kW : 102 EVSEs,Level 2: Unctrl: 200 kW : 30 EVSEs,Level 2: Unctrl: 670 kW : 102 EVSEs,Level 2: LLF: 200 kW : 102 EVSEs
proportion_of_energy_delivered,0.752737,0.996912,0.998591,0.998591
energy_delivered,17608.604672,23320.544956,23359.823145,23359.823145
num_swaps,0.000000,1119.000000,0.000000,0.000000
num_never_charged,0.000000,1.000000,0.000000,0.000000
energy_cost,2631.851150,2915.987677,2826.730582,2919.048218
demand_charge,2471.673600,3070.980000,5817.379684,3070.980000
total_cost,5103.524750,5986.967677,8644.110265,5990.028218
$/kWh,0.289831,0.256725,0.370042,0.256424


From the above table we can see that smart charging using even a simple LLF algorithm has significant benefits over Uncontrolled Level-1 charging in terms of amount of demand met. It also requires far less infrastructure than Uncontrolled Level-2 charging with the same number of EVSEs, and without requiring users to swap spaces mid-day. 